In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
bedDF = pd.read_csv('./project/bed.csv', index_col=0)
bedDF['image'] 

0       ./static/img/bed/bed_/bed_1.png
1       ./static/img/bed/bed_/bed_2.png
2       ./static/img/bed/bed_/bed_3.png
3       ./static/img/bed/bed_/bed_4.png
4       ./static/img/bed/bed_/bed_5.png
                    ...                
94     ./static/img/bed/bed_/bed_98.png
95     ./static/img/bed/bed_/bed_99.png
96    ./static/img/bed/bed_/bed_100.png
97    ./static/img/bed/bed_/bed_101.png
98    ./static/img/bed/bed_/bed_102.png
Name: image, Length: 99, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(bedDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

bedDF['nouns'] = box

100%|██████████| 99/99 [00:26<00:00,  3.70it/s]


In [5]:
bedDF.head(1)

,brand,item,price,image,link,review,nouns
0,노르웨이숲,침대 프레임 원목 깔판 매트리스 받침대 7size,"19,900",./static/img/bed/bed_/bed_1.png,https://ohou.se/productions/125056/selling?aff...,['매트리스 바닥에 아무것도 안깔면 곰팡이 핀다는 말을 듣고 구매했어요. 생각보다 ...,매트리스 바닥 것 곰팡이 말 구매 생각 반 이사 때 휴대 성도 것 배송 정말 새 거...


In [6]:
# 중간 저장
bedDF.to_csv('bedDF.csv')

In [7]:
bedDF = pd.read_csv('./bedDF.csv', index_col=0)
bedDF.head(1)

,brand,item,price,image,link,review,nouns
0,노르웨이숲,침대 프레임 원목 깔판 매트리스 받침대 7size,"19,900",./static/img/bed/bed_/bed_1.png,https://ohou.se/productions/125056/selling?aff...,['매트리스 바닥에 아무것도 안깔면 곰팡이 핀다는 말을 듣고 구매했어요. 생각보다 ...,매트리스 바닥 것 곰팡이 말 구매 생각 반 이사 때 휴대 성도 것 배송 정말 새 거...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(bedDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_bed = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_bed

,가가,가걱,가게,가격,가견,가공,가관,가구,가구라,가구점,...,히히히,힌거,힌데,힌지,힐때,힐링,힐일,힘드셧을텐데,힙겹,힞니
0,0,0,0,13,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,15,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,5,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,8,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,15,0,0,0,15,0,0,...,2,0,0,0,0,0,0,0,0,0
95,0,0,0,8,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,7,0,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,1,17,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# dtm 파일 저장
dtm_bed.to_csv('dtm_bed.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

bedDF = pd.read_csv('./bedDF.csv', index_col=0)
dtm_bed = pd.read_csv('./dtm_bed.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(bedDF['brand'])
item_list = np.array(bedDF['item'])
price_list = np.array(bedDF['price'])
image_list = np.array(bedDF['image'])

result_cosine = make_cosine(dtm_bed, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

인베스코  숨쉬는 침대깔판 플라스틱 바닥깔판 받침대 (높이선택) 28,000 ./static/img/bed/bed_/bed_25.png


In [ ]:
# 위의 코드 함수화
def find_bed(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    bedDF = pd.read_csv('./bedDF.csv', index_col=0)
    dtm_bed = pd.read_csv('./dtm_bed.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(bedDF['brand'])
    item_list = np.array(bedDF['item'])
    price_list = np.array(bedDF['price'])
    image_list = np.array(bedDF['image'])

    result_cosine = make_cosine(dtm_bed, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]